In [1]:
import random
import time
import requests
from bs4 import BeautifulSoup, NavigableString, Tag

BASE = "https://www.trismegistos.org/text/{id3}"

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/124.0 Safari/537.36"
    ),
    "Accept-Language": "en,fr;q=0.9",
}

def extract_language(html: str) -> str | None:
    """
    Tente d'extraire la valeur affichée pour 'Language/script:'.
    On cherche le <span class="semibold">Language/script:</span>
    puis on lit le texte qui suit dans le même <p>.
    """
    soup = BeautifulSoup(html, "lxml")

    # 1) Heuristique robuste : trouver le span portant l'étiquette
    spans = soup.select("span.semibold")
    label_span = None
    for sp in spans:
        if sp.get_text(strip=True).lower() == "language/script:":
            label_span = sp
            break

    if label_span:
        # Valeur la plus fiable : concat des siblings (texte/éléments) après le span
        parts = []
        for sib in label_span.next_siblings:
            if isinstance(sib, NavigableString):
                parts.append(str(sib))
            elif isinstance(sib, Tag):
                parts.append(sib.get_text(" ", strip=True))
        value = " ".join(" ".join(parts).split()).strip(" :\u00a0-")  # normalise espaces
        if value:
            return value

        # Repli : prendre le texte du <p> parent en enlevant l’étiquette
        p = label_span.find_parent("p")
        if p:
            full = " ".join(p.get_text(" ", strip=True).split())
            for prefix in ("Language/script:", "Language/script"):
                if full.startswith(prefix):
                    return full[len(prefix):].strip(" :\u00a0-") or None

    # CSS : #text-details > p:nth-child(4) > span  -> lire le texte du parent <p>
    try:
        sp = soup.select_one("#text-details > p:nth-child(4) > span")
        if sp and sp.get_text(strip=True).lower().startswith("language/script"):
            p = sp.find_parent("p")
            if p:
                full = " ".join(p.get_text(" ", strip=True).split())
                return full.split(":", 1)[1].strip() if ":" in full else None
    except Exception:
        pass

    # XPath équivalents ne sont pas utilisés directement par bs4,
    # mais on a déjà couvert les chemins ci-dessus.
    return None


def check_pages(ids: list[int]) -> list[dict]:
    results = []
    for n in ids:
        id3 = f"{n:03d}"  # zéro-remplissage comme dans 001 … 100
        url = BASE.format(id3=id3)
        row = {"id": n, "id3": id3, "url": url, "status": None, "language": None, "is_greek": None, "error": None}
        try:
            resp = requests.get(url, headers=HEADERS, timeout=15)
            row["status"] = resp.status_code
            if resp.status_code == 200:
                lang = extract_language(resp.text)
                row["language"] = lang
                row["is_greek"] = (lang is not None) and (lang.strip().lower().startswith("greek"))
            else:
                row["error"] = f"HTTP {resp.status_code}"
        except requests.RequestException as e:
            row["error"] = str(e)
        results.append(row)

        # Petite pause polie pour éviter de surcharger le site
        time.sleep(random.uniform(0.6, 1.4))
    return results


if __name__ == "__main__":
    # 20 entiers aléatoires entre 0 et 100 (sans doublons).
    # Si vous voulez autoriser des doublons, remplacez par: [random.randint(0, 100) for _ in range(20)]
    sample_ids = random.sample(range(0, 101), 20)

    data = check_pages(sample_ids)

    # Affichage lisible
    ok = 0
    print(f"{'ID':>3}  {'URL':<45} {'HTTP':<4}  {'Language':<20}  Greek?")
    print("-" * 90)
    for r in data:
        mark = "✓" if r["is_greek"] else ("-" if r["is_greek"] is None else "✗")
        lang = r["language"] or ""
        http = r["status"] if r["status"] is not None else ""
        print(f"{r['id']:>3}  {r['url']:<45} {str(http):<4}  {lang:<20}  {mark}")
        if r["is_greek"]:
            ok += 1
    print("-" * 90)
    print(f"Pages où Language/script == 'Greek' (ou commence par 'Greek') : {ok}/{len(data)}")

 ID  URL                                           HTTP  Language              Greek?
------------------------------------------------------------------------------------------
 82  https://www.trismegistos.org/text/082         200   Greek                 ✓
 38  https://www.trismegistos.org/text/038         200   Demotic               ✗
 15  https://www.trismegistos.org/text/015         200   Greek                 ✓
  3  https://www.trismegistos.org/text/003         200   Greek                 ✓
 25  https://www.trismegistos.org/text/025         200   Demotic               ✗
 77  https://www.trismegistos.org/text/077         200   Greek                 ✓
 16  https://www.trismegistos.org/text/016         200   Greek                 ✓
 66  https://www.trismegistos.org/text/066         200   Demotic / Greek       ✗
 88  https://www.trismegistos.org/text/088         200   Greek                 ✓
 69  https://www.trismegistos.org/text/069         200   Greek                 ✓
 44  https://

In [2]:
!python scrape_trismegistos_geojson.py --start 1500 --end 2000 --out out --delay 0.5 --resume

2025-10-29 10:15:04,284 | INFO | ID 1500 → sauvegardé: out/id_1500.json
2025-10-29 10:15:04,972 | INFO | ID 1501 → sauvegardé: out/id_1501.json
2025-10-29 10:15:05,659 | INFO | ID 1502 → sauvegardé: out/id_1502.json
2025-10-29 10:15:06,348 | INFO | ID 1503 → sauvegardé: out/id_1503.json
2025-10-29 10:15:07,038 | INFO | ID 1504 → sauvegardé: out/id_1504.json
2025-10-29 10:15:07,724 | INFO | ID 1505 → sauvegardé: out/id_1505.json
2025-10-29 10:15:08,403 | INFO | ID 1506 → sauvegardé: out/id_1506.json
2025-10-29 10:15:09,087 | INFO | ID 1507 → sauvegardé: out/id_1507.json
2025-10-29 10:15:09,761 | INFO | ID 1508 → sauvegardé: out/id_1508.json
2025-10-29 10:15:10,449 | INFO | ID 1509 → sauvegardé: out/id_1509.json
2025-10-29 10:15:11,138 | INFO | ID 1510 → sauvegardé: out/id_1510.json
2025-10-29 10:15:11,811 | INFO | ID 1511 → sauvegardé: out/id_1511.json
2025-10-29 10:15:12,484 | INFO | ID 1512 → sauvegardé: out/id_1512.json
2025-10-29 10:15:13,172 | INFO | ID 1513 → sauvegardé: out/id_15

In [6]:
import folium
import json
import os

# --- Configuration ---
# Chemin vers le dossier contenant les fichiers JSON
JSON_DATA_PATH = '/content/out'
OUTPUT_MAP_FILE = 'ma_carte_avec_points.html'


# --- Création de la carte ---
# On centre la carte sur un point neutre avec un faible zoom pour voir le monde entier
m = folium.Map(location=[20, 0], zoom_start=2)


# --- Lecture des fichiers JSON et ajout des points ---
print(f"Recherche de fichiers JSON dans le dossier '{JSON_DATA_PATH}'...")
points_ajoutes = 0

# Parcourir tous les fichiers dans le dossier spécifié
for filename in os.listdir(JSON_DATA_PATH):
    if filename.endswith('.json'):
        filepath = os.path.join(JSON_DATA_PATH, filename)

        # Le bloc try/except est idéal pour ignorer les fichiers qui n'ont pas la bonne structure
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f)

                # --- NOUVELLE LOGIQUE D'EXTRACTION (GeoJSON) ---
                # 1. Accéder aux coordonnées via le chemin "geometry" -> "coordinates"
                coords_lon_lat = data['geometry']['coordinates']

                # 2. VÉRIFICATION IMPORTANTE : Inverser les coordonnées
                #    GeoJSON est [longitude, latitude]
                #    Folium attend [latitude, longitude]
                #    On vérifie aussi qu'on a bien une paire de coordonnées
                if isinstance(coords_lon_lat, list) and len(coords_lon_lat) == 2:
                    coords_lat_lon = [coords_lon_lat[1], coords_lon_lat[0]]
                else:
                    # Si le format est inattendu, on lève une exception pour que le 'except' l'attrape
                    raise ValueError("Format de coordonnées invalide")

                # --- Extraire les informations pour la popup (nom, pays, etc.) ---
                # Dans GeoJSON, ces infos sont généralement dans un objet "properties"
                # On utilise .get() pour éviter les erreurs si "properties" n'existe pas
                properties = data.get('properties', {}) # Renvoie un dictionnaire vide si 'properties' n'existe pas
                # Ajouter un marqueur (un petit point) à la carte
                folium.Marker(
                    location=coords_lat_lon,
                    popup=popup_text,
                    icon=folium.Icon(color='green', icon='info-sign')
                ).add_to(m)

                print(f"-> Point ajouté pour le fichier '{filename}' ({city_name})")
                points_ajoutes += 1

        except (KeyError, TypeError, ValueError) as e:
            # Ce bloc s'exécute si une clé manque ('geometry', 'coordinates') ou si le format est mauvais
            # C'est exactement ce qu'il faut pour ignorer les fichiers sans coordonnées
            print(f"--- Fichier ignoré '{filename}': Coordonnées non trouvées ou format incorrect.")
        except Exception as e:
            # Pour attraper toute autre erreur imprévue
            print(f"Erreur inattendue avec le fichier '{filename}': {e}")


# --- Sauvegarde de la carte ---
if points_ajoutes > 0:
    m.save(OUTPUT_MAP_FILE)
    print(f"\nCarte générée avec succès ! {points_ajoutes} points ont été ajoutés.")
    print(f"Ouvrez le fichier '{OUTPUT_MAP_FILE}' dans votre navigateur.")
else:
    print("\nAucun point n'a été ajouté. Vérifiez le format de vos fichiers JSON et les chemins d'accès dans le script.")

Recherche de fichiers JSON dans le dossier '/content/out'...
--- Fichier ignoré 'id_1568.json': Coordonnées non trouvées ou format incorrect.
--- Fichier ignoré 'id_1708.json': Coordonnées non trouvées ou format incorrect.
-> Point ajouté pour le fichier 'id_1726.json' (Lieu inconnu)
--- Fichier ignoré 'id_1938.json': Coordonnées non trouvées ou format incorrect.
--- Fichier ignoré 'id_1973.json': Coordonnées non trouvées ou format incorrect.
--- Fichier ignoré 'id_1523.json': Coordonnées non trouvées ou format incorrect.
-> Point ajouté pour le fichier 'id_1915.json' (Lieu inconnu)
--- Fichier ignoré 'id_1926.json': Coordonnées non trouvées ou format incorrect.
--- Fichier ignoré 'id_1968.json': Coordonnées non trouvées ou format incorrect.
--- Fichier ignoré 'id_1669.json': Coordonnées non trouvées ou format incorrect.
--- Fichier ignoré 'id_1640.json': Coordonnées non trouvées ou format incorrect.
-> Point ajouté pour le fichier 'id_1565.json' (Lieu inconnu)
--- Fichier ignoré 'id_1